In [3]:
import math
import os

import numpy as np
import pandas as pd

import dask
from dask.distributed import Client
import dask.array as da
import dask.dataframe as dd

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:34293 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 134.93 GB


In [6]:
data_path = "/sps/lsst/data/desc/gcr_rootdir/xgal/skysim/skysim5000_v1.1.1_parquet"

In [9]:
columns = [ "ra", "dec", "mag_u", "mag_g", "mag_r", "mag_i", "mag_z", "mag_y", "redshift"]

In [10]:
%%time
df = dd.read_parquet(data_path, columns=columns, engine='pyarrow', kwargs={'dataset': {'use_legacy_dataset': False}})

CPU times: user 10.6 s, sys: 779 ms, total: 11.4 s
Wall time: 10.6 s


In [11]:
%%time
len(df)

8503061280

In [12]:
df.head()

,ra,dec,mag_u,mag_g,mag_r,mag_i,mag_z,mag_y,redshift
0,0.432719,0.046171,21.098677,20.032227,19.708700,19.542538,19.488430,19.443745,0.020688
1,2.040416,-0.534684,17.333044,16.413969,15.929143,15.571618,15.348542,15.232266,0.006596
2,0.944323,-0.734390,18.756887,17.835205,17.638680,17.540789,17.521290,17.487024,0.018310
3,0.502705,-0.147265,20.579208,19.662645,19.219460,18.910147,18.736389,18.631491,0.020962
4,2.007109,-0.019202,18.627811,17.774061,17.404118,17.080204,16.880186,16.768349,0.019156


In [20]:
cut = (df.redshift > 1.0) & (df.redshift < 1.01)

In [21]:
cut

Dask Series Structure:
npartitions=1568
    bool
     ...
    ... 
     ...
     ...
Name: redshift, dtype: bool
Dask Name: and_, 7840 tasks

In [22]:
df2 = df[cut].compute()

In [18]:
df2.head()

,ra,dec,mag_u,mag_g,mag_r,mag_i,mag_z,mag_y,redshift
397607,1.042379,-0.782734,24.980705,24.907329,24.394134,23.643894,23.025331,22.889061,1.001209
397608,1.041526,-0.784054,28.084831,27.795216,27.166285,26.362562,25.737995,25.557953,1.000656
397617,0.536179,-0.302533,27.145730,27.188059,27.056357,26.201157,25.978373,25.887265,1.008801
397621,0.753302,-0.578448,27.198244,26.833908,26.182133,25.366656,24.746761,24.637648,1.008924
397626,0.389237,-0.325381,26.453023,26.455196,26.259038,25.748531,25.326802,25.317304,1.004504


In [19]:
%%time
len(df2)

29115056

In [23]:
df2

,ra,dec,mag_u,mag_g,mag_r,mag_i,mag_z,mag_y,redshift
397607,1.042379,-0.782734,24.980705,24.907329,24.394134,23.643894,23.025331,22.889061,1.001209
397608,1.041526,-0.784054,28.084831,27.795216,27.166285,26.362562,25.737995,25.557953,1.000656
397617,0.536179,-0.302533,27.145730,27.188059,27.056357,26.201157,25.978373,25.887265,1.008801
397621,0.753302,-0.578448,27.198244,26.833908,26.182133,25.366656,24.746761,24.637648,1.008924
397626,0.389237,-0.325381,26.453023,26.455196,26.259038,25.748531,25.326802,25.317304,1.004504
...,...,...,...,...,...,...,...,...,...
972308,59.074570,-88.124821,26.134877,26.243435,26.312003,26.053946,25.766404,25.804101,1.008084
972312,45.901874,-87.493213,27.100227,26.945985,26.533807,25.899242,25.380029,25.339443,1.008251
972346,60.249298,-89.620737,27.001023,27.209193,26.951203,26.502192,26.076323,26.000008,1.008500
972399,83.211816,-89.042835,25.662015,25.809127,25.953975,25.832864,25.649040,25.708820,1.008455


In [1]:
def limit_numpy(nthreads=4):
     """ """
     import os
     threads = str(nthreads)
     print(f"threads {threads}")
     os.environ["NUMEXPR_NUM_THREADS"] = threads
     os.environ["OMP_NUM_THREADS"] = threads
     os.environ["OPENBLAS_NUM_THREADS"] = threads
     os.environ["MKL_NUM_THREADS"] = threads
     os.environ["VECLIB_MAXIMUM_THREADS"] = threads
limit_numpy(4)
import dask
from dask_jobqueue import SGECluster
from dask.distributed import Client
import dask.dataframe as dd

threads 4


In [2]:
cluster = SGECluster(name="dask-worker",  walltime="02:00:00", memory='5GB', death_timeout=120, project="P_lsst", resource_spec='sps=1', cores=1, processes=1)
cluster.scale(100)
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://134.158.240.242:37697 Dashboard: http://134.158.240.242:8787/status,Cluster Workers: 100 Cores: 100 Memory: 499.51 GB


In [6]:
%%time
data_path = "/sps/lsst/data/desc/gcr_rootdir/xgal/skysim/skysim5000_v1.1.1_parquet"
columns = [ "ra", "dec", "mag_u", "mag_g", "mag_r", "mag_i", "mag_z", "mag_y", "redshift"]
df = dd.read_parquet(data_path, columns=columns, engine='pyarrow', kwargs={'dataset': {'use_legacy_dataset': False}})

CPU times: user 12.4 s, sys: 834 ms, total: 13.2 s
Wall time: 12.8 s


In [ ]:
%%time
cut = (df.redshift > 1.0) & (df.redshift < 1.01)
df2 = df[cut].compute()